In [74]:
#!/usr/bin/env python

import sys
sys.path.insert(0,'/Users/cmdb/projects/ConfocalAnalysis')
sys.path.insert(0,'/Users/babygirljohnston/Documents/ConfocalAnalysis/ConfocalAnalysis')
sys.path.insert(0,'/Users/babygirljohnston/Documents/Confocal_Xiuqi/scikit-image')

from czifile import CziFile
import numpy as np
from math import sqrt
from skimage.feature import blob_log
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.cluster import DBSCAN
from sklearn import metrics
from mpl_toolkits.mplot3d import Axes3D
import operator
import matplotlib.cm as cm

%matplotlib qt 
#interactive mode of plt

#%matplotlib inline 
# plt in line with notebook
'''Global variables'''
colomap=["Blues_r","gray","Greens_r","Reds_r"] # colormap for channels
labell=['yellow','yellow','yellow','yellow'] # label color for blobs

'''Setup tips'''
# pip install scikit-image # if not working, compile from source
# pip install -U scikit-learn
# If weird errors happen, try update the packages except 'pip install setuptools==33.1.1'
# different mpl might have version clahes, remember to clean the older version.
# I manually replace the /System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/mpl_toolkits
# with the newer version from /Library/Python/2.7/site-packages/mpl_toolkits

"""Useful Functions"""

def GetOneChannel( czi_file, channel ):
# this function takes the whole .czi file and return a single channel .czi
    stacknumber=czi_file.shape[3]
    OneChannel=[czi_file[0,channel,0,index].T[0].T for index in range(stacknumber)]
    return OneChannel

def FindBlobs( OneChannel,thresh=0.2 ):
# this function takes output from GetOneChannel (a single channel .czi file input)
# return all the blobs in a dict & a structured list & a flattened list
# Dict    # "Y,X,Z"->[radius, intensity]
# struct. List    # [[[Y,X,Z1,radius,intensity],[Y,X,Z1,...]...],...,[[Y,X,Zn,...],...]],np.array
# flat. List    # [[Y1,X1,Z1],[Y2,X2,Z2],[Y3,X3,Z3],...,[Yn,X3,Z3]]
  
    blobs_dict={}
    blobsAll=[]
    blobsList=[]
    stacknumber=len(OneChannel)
    
    for z in range(stacknumber):
        planeblobs=[]
        """Blob Recognition Parameters here"""
        log = blob_log(OneChannel[z], max_sigma=30, num_sigma=10,min_sigma =3, threshold=thresh)
        # Radius of the blob
        log[:, 2] = log[:, 2] * sqrt(2)
        # intensity
        blob_intensity=np.zeros(len(log[:,0]))
        
        for i in range(len(log[:,0])):
            blob_intensity[i]=OneChannel[z][int(log[i,0]),int(log[i,1])]
        
        for i in range(len(log[:,2])):
            position=np.append(log[i,0:2],z)
            position=map(int,position)
            blobs_dict["{},{},{}".format(position[0],position[1],position[2])]=[log[i,2],blob_intensity[i]]
            blobsfeature=[log[i,0],log[i,1],z,log[i,2],blob_intensity[i]]
            print blobsfeature
            planeblobs.append(blobsfeature)
            blobsList.append(position)
        
        blobsAll.append(planeblobs)
    
    blobsAll=np.array(blobsAll)
    blobsList=np.array(blobsList)
    
    return blobs_dict,blobsAll,blobsList



def GetClustersOneChannel( blobsList ):
# this function takes the dict and flattened list from FindBlobs
# return the clusters
###  [[[cluster1_dot1],[cluster1_dot2],...],
#     [[cluster2_dot1],[cluster2_dot2]...],
#     [...],...,
#     [[clusterN_dot1],[clusterN_dot2]] ]

    """3D clustering of Blobs"""
    db = DBSCAN(eps=2, min_samples=2).fit(blobsList)  # eps is the max distance between 2 dots in one cluster
    labels = db.labels_
    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

    print('Estimated number of clusters: %d' % n_clusters_)
#     print("Silhouette Coefficient: %0.3f"
#           % metrics.silhouette_score(blobsList, labels))
    
    """Group up the dots within a cluster"""
    clusters=[]
    for i in range(n_clusters_):
        cluster=[]
        ## Extract X,Y,Z location of the points based on clusters
        positions=np.argwhere(labels==i)
        for j in range(len(positions)):
            cluster.append(blobsList[positions[j]])
        #print cluster
        clusters.append(cluster)
    
    return clusters

def SignalPositionsOneChannel( clusters,blobs_dict ):
# This function takes the clusters List from GetClustersOneChannel
# return the calculated signal position in this one channel all stacks
# [[Y,X,Z,avgIntensity],[Y,X,Z,avgIntensity],...,[Y,X,Z,avgIntensity]]
    """Find the actual signal location"""
    signals=[]
    for i in range(len(clusters)):
        signal=[]
        dots=[]
        ## extract the intensity of the dot in the cluster (from the dictionary)
        for j in range(len(clusters[i])):
            #print clusters[i][j][0]
            dot=clusters[i][j][0]
            intensity=blobs_dict["{},{},{}".format(clusters[i][j][0][0],clusters[i][j][0][1],clusters[i][j][0][2])][1]
            dot=np.append(dot,intensity)
            dots.append(dot)
        # Calculating the actual dot location
        intensity_sum=sum(dots[index][3] for index in range(len(dots)))
        avgIntensity=intensity_sum/len(dots)
        y=sum(dots[index][0]*dots[index][3]/intensity_sum for index in range(len(dots)))
        x=sum(dots[index][1]*dots[index][3]/intensity_sum for index in range(len(dots)))
        z=sum(dots[index][2]*dots[index][3]/intensity_sum for index in range(len(dots)))
        signal=[y,x,z,avgIntensity]
        signals.append(signal)
    signals=np.array(signals)
    return signals

def ToActualScale(list):
# This function takes a [n_sample,[Y,X,Z,Intensity]] LIST
# convert to the actual scale of the coordinates
    """Adjust Parameters"""
    # 63X objective, unit: um
    x_scale=0.099
    y_scale=0.099
    z_scale=0.2
    actual=np.zeros((len(list),len(list[0])))
    for i in range(len(list)):
        actual[i][0]=list[i][0]*y_scale
        actual[i][1]=list[i][1]*x_scale
        actual[i][2]=list[i][2]*z_scale
        if len(list[0])==4:
            actual[i][3]=list[i][3]
    return actual

def GetDistance(element1,element2):
# This function takes two [Y,X,Z]
# return euclidean distance
    distance=sqrt((element1[0]-element2[0])**2+(element1[1]-element2[1])**2+(element1[2]-element2[2])**2)
    return distance

def FindPairs( signals1,signals2,limit=1.2 ):
# This function takes 2 signals-Lists generated in SignalPositionsOneChannel
# return pairs  (LIST)
# [[sig1_idx1,sig2_idx1,dist],[sig1_idx2,sig2_idx2,dist],...,[sig1_idxN,sig2_idxN,dist]]
    pairs=[]
    signals1=ToActualScale(signals1)
    signals2=ToActualScale(signals2)
    #print signals1
    #print signals2
    
    # Brute-Force calculate the distance matrix between the two signal groups
    # 
    row_num= len(signals1)
    col_num= len(signals2)
    matrix=np.zeros((row_num,col_num))
    for i in range(row_num):
        for j in range(col_num):
            matrix[i][j]=GetDistance(signals1[i],signals2[j])
    while 1:
        min_idx=matrix.argmin()
        row=min_idx/col_num
        col=min_idx%col_num
        distance=matrix[row][col]
        if distance > limit:
            break
        pairs.append([row,col,distance])
        matrix[row,:]=100
        matrix[:,col]=100
    return pairs


def PlotOriginal(image_arrays,channel,stack):
    # This function takes the czifile from the reader line, specific channel, stack and colormap
    # plot the image
    images=GetOneChannel(image_arrays,channel)
    fig=plt.figure(figsize=(20,20))
    mapp=plt.imshow(images[stack],cmap=colomap[channel])
    cbar = fig.colorbar(mapp, ticks=[0, 65535])
    cbar.ax.set_yticklabels(['0', '65535'])
    plt.title("Channel {}, Stack {}".format(channel,stack),fontsize=20)
    return mapp

def PlotCheckBlobs(OneChannel, blobsAll, channel, stack):
    # Take Single channel all stack, blobsAll from FindBlobs, stack,label
    # Plot the specific image on that stack, together with the blobs.
    fig, axes = plt.subplots(1, 1, sharex=True, sharey=True,figsize=(20,20))
    mapp=plt.imshow(OneChannel[stack],cmap=colomap[channel])

    cbar = fig.colorbar(mapp, ticks=[0, 65535])
    cbar.ax.set_yticklabels(['0', '65535'])

    log = blobsAll[stack]
    #print log
    for blob in log:
        #print blob
        y, x, z,radius,intensity = blob
        c = plt.Circle((x, y), radius, color=labell[channel], linewidth=2, fill=False)   
        axes.add_patch(c)
    
    plt.title("Channel {}, Stack {}".format(channel,stack),fontsize=20)
    
    return mapp

def PlotCheckClusters(clusters):
    # This function takes the clusters & plot each cluster in a different color to inspect
    colors = cm.rainbow(np.linspace(0, 1, len(clusters_ss)))
    fig = plt.figure(figsize=(20,20))
    ax = fig.add_subplot(111, projection='3d')
    for cluster, c in zip(clusters_ss, colors):
        #print cluster
        y,x,z=np.array([cluster[index] for index in range(len(cluster))]).T
        ax.scatter(x, y, z,color=c)
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    return 0

def PlotPairs3D(pairs_coords, threshold=0):
    # This function takes the actual pair_coord, 
    #[[[y1,x1,z1],[y1_,x1_,z1_]],
    # [[y2,x2,z2],[y2_,x2_,z2_]],
    # ...
    #]
    # takes the min threshold and plot the pairs above the threshold in 3D
    
    selected=[]
    for pair in pairs_coords:
        #print pair
        distance=GetDistance(pair[0],pair[1])
        if distance>threshold:
            selected.append(pair)

    y_s,x_s,z_s = np.array([selected[index][0] for index in range(len(selected))]).T
    y_k,x_k,z_k = np.array([selected[index][1] for index in range(len(selected))]).T
    fig = plt.figure(figsize=(20,20))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x_s, y_s, z_s,color='green')
    ax.scatter(x_k, y_k, z_k,color='red')
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    ax.set_zlim([-1,7])
    plt.title("3D Plot of Signal Pairs with Distance above {} um".format(threshold),fontsize=20)
    return ax

def PlotPairsImage(image_arrays,channel, stack, pair_coord_pic, threshold=0):
    OneChannel_back=GetOneChannel(image_arrays,channel)
    # len(OneChannel_back) # stack number

    # Show the background channel
    fig, axes = plt.subplots(1, 1, sharex=True, sharey=True,figsize=(20,20))
    mapp=plt.imshow(OneChannel_back[stack],cmap=colomap[channel])

    cbar = fig.colorbar(mapp, ticks=[0, 65535])
    cbar.ax.set_yticklabels(['0', '65535'])

    #print pair_coord_pic

    selected=[]
    for pair in pair_coord_pic:
        #print pair
        distance=GetDistance(pair[0],pair[1])
        if distance<threshold:  # only take the pairs with distance above threshold
            continue
        if pair[0][2]>stack+0.5 or pair[0][2]<stack-0.5: # take the pairs near the stack
            continue

        selected.append(pair)

    #print selected

    y_s,x_s,z_s = np.array([selected[index][0] for index in range(len(selected))]).T
    y_k,x_k,z_k = np.array([selected[index][1] for index in range(len(selected))]).T

    axes.scatter(x_s, y_s,color='green')
    axes.scatter(x_k, y_k,color='red')
    axes.set_xlabel('X Label')
    axes.set_ylabel('Y Label')

    plt.title("Channel {}, Stack {}".format(channel,stack),fontsize=20)
    
    return axes

def PlotPairsProjected(image_arrays,channel,stack,pair_coord_pic,threshold=0):

    OneChannel_back=GetOneChannel(image_arrays,channel)

    # Show background
    fig, axes = plt.subplots(1, 1, sharex=True, sharey=True,figsize=(20,20))
    mapp=plt.imshow(OneChannel_back[stack],cmap=colomap[channel])

    cbar = fig.colorbar(mapp, ticks=[0, 65535])
    cbar.ax.set_yticklabels(['0', '65535'])

    selected=[]
    for pair in pair_coord_pic:
        #print pair
        distance=GetDistance(pair[0],pair[1])
        if distance>threshold:
            selected.append(pair)

    # plot all the long pairs
    y_s,x_s,z_s = np.array([selected[index][0] for index in range(len(selected))]).T
    y_k,x_k,z_k = np.array([selected[index][1] for index in range(len(selected))]).T

    axes.scatter(x_s, y_s,color='green')
    axes.scatter(x_k, y_k,color='red')

    plt.title("Channel {}, Stack {}".format(channel,stack),fontsize=20)

    
    return 

In [3]:
""" Image Input """
with CziFile('/Users/babygirljohnston/Documents/Confocal_Xiuqi/Image_1.czi') as czi:
    image_arrays=czi.asarray()
# 63X objective, unit: um

In [4]:
## Plot the original image of any channel, any stack

channel=3
stack=7

PlotOriginal(image_arrays,channel,stack)
# plt.plot((x1, x2), (y1, y2), 'k-') #to draw a line from the point (x1, y1) to the point (x2, y2)

In [5]:
"""Image Processing -- (time consuming: several minutes)""" 
# Find blobs in one channel image image for ss
OneChannel_ss=GetOneChannel(image_arrays,2)
blobs_dict_ss,blobsAll_ss,blobsList_ss=FindBlobs(OneChannel_ss,0.1)

# Find blobs in one channel image stack for klu
OneChannel_klu=GetOneChannel(image_arrays,3)
blobs_dict_klu,blobsAll_klu,blobsList_klu=FindBlobs(OneChannel_klu,0.1)

[1021.0, 683.0, 0, 4.2426406871192857, 21697.0]
[1020.0, 755.0, 0, 4.2426406871192857, 41144.0]
[1010.0, 906.0, 0, 4.2426406871192857, 26610.0]
[1006.0, 503.0, 0, 4.2426406871192857, 39789.0]
[1004.0, 940.0, 0, 4.2426406871192857, 34336.0]
[1003.0, 598.0, 0, 4.2426406871192857, 22649.0]
[1001.0, 905.0, 0, 4.2426406871192857, 28661.0]
[999.0, 219.0, 0, 4.2426406871192857, 35991.0]
[993.0, 274.0, 0, 4.2426406871192857, 31121.0]
[988.0, 250.0, 0, 4.2426406871192857, 11662.0]
[978.0, 921.0, 0, 4.2426406871192857, 45000.0]
[973.0, 252.0, 0, 4.2426406871192857, 21179.0]
[972.0, 292.0, 0, 4.2426406871192857, 21992.0]
[971.0, 545.0, 0, 4.2426406871192857, 30540.0]
[971.0, 415.0, 0, 4.2426406871192857, 20323.0]
[968.0, 439.0, 0, 4.2426406871192857, 43178.0]
[963.0, 429.0, 0, 4.2426406871192857, 24606.0]
[963.0, 331.0, 0, 4.2426406871192857, 21513.0]
[961.0, 318.0, 0, 4.2426406871192857, 22640.0]
[960.0, 283.0, 0, 4.2426406871192857, 34663.0]
[958.0, 129.0, 0, 4.2426406871192857, 28017.0]
[955.0

[1020.0, 756.0, 1, 4.2426406871192857, 42607.0]
[1010.0, 907.0, 1, 4.2426406871192857, 28182.0]
[1009.0, 366.0, 1, 4.2426406871192857, 25537.0]
[1006.0, 503.0, 1, 4.2426406871192857, 40587.0]
[1004.0, 940.0, 1, 4.2426406871192857, 33651.0]
[1002.0, 598.0, 1, 4.2426406871192857, 18902.0]
[999.0, 219.0, 1, 4.2426406871192857, 41612.0]
[978.0, 921.0, 1, 4.2426406871192857, 45211.0]
[972.0, 293.0, 1, 4.2426406871192857, 30559.0]
[971.0, 545.0, 1, 4.2426406871192857, 18233.0]
[970.0, 415.0, 1, 4.2426406871192857, 24149.0]
[967.0, 439.0, 1, 4.2426406871192857, 38075.0]
[963.0, 331.0, 1, 4.2426406871192857, 22607.0]
[961.0, 318.0, 1, 4.2426406871192857, 27027.0]
[949.0, 755.0, 1, 4.2426406871192857, 23067.0]
[943.0, 112.0, 1, 4.2426406871192857, 15575.0]
[941.0, 240.0, 1, 4.2426406871192857, 44923.0]
[939.0, 478.0, 1, 4.2426406871192857, 28869.0]
[937.0, 174.0, 1, 4.2426406871192857, 19245.0]
[934.0, 379.0, 1, 4.2426406871192857, 35999.0]
[931.0, 928.0, 1, 4.2426406871192857, 27169.0]
[928.0,

[1019.0, 756.0, 2, 4.2426406871192857, 36982.0]
[1019.0, 625.0, 2, 4.2426406871192857, 25459.0]
[1009.0, 450.0, 2, 4.2426406871192857, 29154.0]
[1005.0, 504.0, 2, 4.2426406871192857, 26917.0]
[1004.0, 940.0, 2, 4.2426406871192857, 27627.0]
[1002.0, 599.0, 2, 4.2426406871192857, 16679.0]
[1000.0, 471.0, 2, 4.2426406871192857, 21348.0]
[998.0, 287.0, 2, 4.2426406871192857, 32126.0]
[995.0, 879.0, 2, 4.2426406871192857, 26239.0]
[994.0, 408.0, 2, 4.2426406871192857, 26144.0]
[985.0, 896.0, 2, 4.2426406871192857, 31541.0]
[985.0, 190.0, 2, 4.2426406871192857, 20771.0]
[977.0, 921.0, 2, 4.2426406871192857, 22754.0]
[972.0, 293.0, 2, 4.2426406871192857, 26709.0]
[969.0, 415.0, 2, 4.2426406871192857, 31924.0]
[967.0, 440.0, 2, 4.2426406871192857, 22439.0]
[961.0, 319.0, 2, 4.2426406871192857, 33478.0]
[949.0, 437.0, 2, 4.2426406871192857, 22661.0]
[941.0, 241.0, 2, 4.2426406871192857, 42255.0]
[939.0, 479.0, 2, 4.2426406871192857, 27008.0]
[934.0, 380.0, 2, 4.2426406871192857, 36725.0]
[932.0

[1023.0, 624.0, 3, 8.4852813742385713, 9659.0]
[1019.0, 254.0, 3, 4.2426406871192857, 35713.0]
[1017.0, 495.0, 3, 4.2426406871192857, 21565.0]
[1013.0, 566.0, 3, 4.2426406871192857, 13832.0]
[1008.0, 526.0, 3, 4.2426406871192857, 20869.0]
[1008.0, 449.0, 3, 4.2426406871192857, 29630.0]
[1008.0, 370.0, 3, 4.2426406871192857, 25990.0]
[1002.0, 599.0, 3, 4.2426406871192857, 28129.0]
[1000.0, 470.0, 3, 4.2426406871192857, 55613.0]
[998.0, 287.0, 3, 4.2426406871192857, 28998.0]
[996.0, 253.0, 3, 4.2426406871192857, 33978.0]
[995.0, 879.0, 3, 4.2426406871192857, 24559.0]
[994.0, 408.0, 3, 4.2426406871192857, 27207.0]
[988.0, 472.0, 3, 4.2426406871192857, 29001.0]
[985.0, 896.0, 3, 4.2426406871192857, 24530.0]
[978.0, 408.0, 3, 4.2426406871192857, 24050.0]
[970.0, 417.0, 3, 4.2426406871192857, 23252.0]
[969.0, 977.0, 3, 4.2426406871192857, 15118.0]
[966.0, 425.0, 3, 4.2426406871192857, 22762.0]
[961.0, 319.0, 3, 4.2426406871192857, 26118.0]
[947.0, 911.0, 3, 4.2426406871192857, 27785.0]
[941.

[1023.0, 625.0, 4, 8.4852813742385713, 10967.0]
[1023.0, 254.0, 4, 8.4852813742385713, 3240.0]
[1017.0, 496.0, 4, 4.2426406871192857, 32934.0]
[1017.0, 367.0, 4, 4.2426406871192857, 37452.0]
[1012.0, 566.0, 4, 4.2426406871192857, 23562.0]
[1008.0, 450.0, 4, 4.2426406871192857, 27849.0]
[1008.0, 370.0, 4, 4.2426406871192857, 36364.0]
[1002.0, 599.0, 4, 4.2426406871192857, 24900.0]
[1000.0, 471.0, 4, 4.2426406871192857, 57622.0]
[998.0, 314.0, 4, 4.2426406871192857, 35894.0]
[996.0, 253.0, 4, 4.2426406871192857, 40929.0]
[994.0, 409.0, 4, 4.2426406871192857, 19251.0]
[988.0, 472.0, 4, 4.2426406871192857, 16760.0]
[985.0, 896.0, 4, 4.2426406871192857, 30887.0]
[977.0, 409.0, 4, 4.2426406871192857, 34397.0]
[976.0, 992.0, 4, 4.2426406871192857, 24744.0]
[969.0, 978.0, 4, 4.2426406871192857, 27574.0]
[966.0, 425.0, 4, 4.2426406871192857, 34710.0]
[958.0, 980.0, 4, 4.2426406871192857, 23080.0]
[958.0, 396.0, 4, 4.2426406871192857, 30123.0]
[947.0, 911.0, 4, 4.2426406871192857, 21553.0]
[947.

...

[1005.0, 332.0, 20, 4.2426406871192857, 14910.0]
[997.0, 986.0, 20, 4.2426406871192857, 25087.0]
[978.0, 856.0, 20, 4.2426406871192857, 26518.0]
[977.0, 666.0, 20, 4.2426406871192857, 27889.0]
[968.0, 375.0, 20, 4.2426406871192857, 39829.0]
[967.0, 616.0, 20, 4.2426406871192857, 29287.0]
[965.0, 552.0, 20, 4.2426406871192857, 58358.0]
[957.0, 729.0, 20, 4.2426406871192857, 29174.0]
[947.0, 511.0, 20, 4.2426406871192857, 34649.0]
[944.0, 1023.0, 20, 4.2426406871192857, 23429.0]
[944.0, 291.0, 20, 4.2426406871192857, 41843.0]
[925.0, 615.0, 20, 4.2426406871192857, 25243.0]
[923.0, 443.0, 20, 4.2426406871192857, 51252.0]
[912.0, 767.0, 20, 4.2426406871192857, 28006.0]
[912.0, 653.0, 20, 4.2426406871192857, 26211.0]
[911.0, 171.0, 20, 4.2426406871192857, 44837.0]
[910.0, 815.0, 20, 4.2426406871192857, 26466.0]
[891.0, 621.0, 20, 4.2426406871192857, 22263.0]
[889.0, 481.0, 20, 4.2426406871192857, 29865.0]
[888.0, 530.0, 20, 4.2426406871192857, 18876.0]
[887.0, 126.0, 20, 4.2426406871192857,

[1005.0, 332.0, 21, 4.2426406871192857, 27621.0]
[977.0, 855.0, 21, 4.2426406871192857, 20747.0]
[977.0, 666.0, 21, 4.2426406871192857, 44993.0]
[969.0, 375.0, 21, 4.2426406871192857, 61471.0]
[967.0, 616.0, 21, 4.2426406871192857, 32898.0]
[965.0, 551.0, 21, 4.2426406871192857, 60553.0]
[958.0, 825.0, 21, 4.2426406871192857, 24519.0]
[947.0, 511.0, 21, 4.2426406871192857, 54578.0]
[944.0, 1023.0, 21, 4.2426406871192857, 30090.0]
[941.0, 513.0, 21, 4.2426406871192857, 22699.0]
[930.0, 551.0, 21, 4.2426406871192857, 23258.0]
[925.0, 614.0, 21, 4.2426406871192857, 21680.0]
[923.0, 443.0, 21, 4.2426406871192857, 65535.0]
[913.0, 767.0, 21, 4.2426406871192857, 21254.0]
[913.0, 653.0, 21, 4.2426406871192857, 35558.0]
[911.0, 171.0, 21, 4.2426406871192857, 27441.0]
[910.0, 815.0, 21, 4.2426406871192857, 39100.0]
[893.0, 950.0, 21, 4.2426406871192857, 12516.0]
[890.0, 620.0, 21, 4.2426406871192857, 18218.0]
[889.0, 481.0, 21, 4.2426406871192857, 30584.0]
[887.0, 126.0, 21, 4.2426406871192857,

[1023.0, 696.0, 22, 4.2426406871192857, 14960.0]
[1010.0, 365.0, 22, 4.2426406871192857, 35914.0]
[1004.0, 332.0, 22, 4.2426406871192857, 15779.0]
[977.0, 856.0, 22, 4.2426406871192857, 15379.0]
[976.0, 666.0, 22, 4.2426406871192857, 43253.0]
[968.0, 375.0, 22, 4.2426406871192857, 54271.0]
[967.0, 616.0, 22, 4.2426406871192857, 22680.0]
[964.0, 552.0, 22, 4.2426406871192857, 27986.0]
[957.0, 825.0, 22, 4.2426406871192857, 33140.0]
[952.0, 908.0, 22, 4.2426406871192857, 23079.0]
[947.0, 99.0, 22, 4.2426406871192857, 25723.0]
[945.0, 511.0, 22, 4.2426406871192857, 14895.0]
[943.0, 1023.0, 22, 4.2426406871192857, 36222.0]
[939.0, 900.0, 22, 4.2426406871192857, 21650.0]
[929.0, 551.0, 22, 4.2426406871192857, 30051.0]
[925.0, 615.0, 22, 4.2426406871192857, 23850.0]
[922.0, 443.0, 22, 4.2426406871192857, 29846.0]
[912.0, 653.0, 22, 4.2426406871192857, 30679.0]
[911.0, 171.0, 22, 4.2426406871192857, 29344.0]
[909.0, 815.0, 22, 4.2426406871192857, 16825.0]
[892.0, 950.0, 22, 4.2426406871192857

[1023.0, 696.0, 23, 4.2426406871192857, 19319.0]
[1010.0, 365.0, 23, 4.2426406871192857, 24625.0]
[991.0, 504.0, 23, 4.2426406871192857, 26174.0]
[977.0, 856.0, 23, 4.2426406871192857, 15613.0]
[976.0, 667.0, 23, 4.2426406871192857, 28450.0]
[968.0, 376.0, 23, 4.2426406871192857, 24476.0]
[957.0, 826.0, 23, 4.2426406871192857, 23822.0]
[952.0, 908.0, 23, 4.2426406871192857, 8991.0]
[947.0, 100.0, 23, 4.2426406871192857, 20753.0]
[946.0, 153.0, 23, 4.2426406871192857, 33191.0]
[944.0, 1023.0, 23, 4.2426406871192857, 25934.0]
[938.0, 900.0, 23, 4.2426406871192857, 17314.0]
[929.0, 551.0, 23, 4.2426406871192857, 24287.0]
[925.0, 615.0, 23, 4.2426406871192857, 23263.0]
[912.0, 654.0, 23, 4.2426406871192857, 26366.0]
[892.0, 950.0, 23, 4.2426406871192857, 14538.0]
[887.0, 126.0, 23, 4.2426406871192857, 29961.0]
[880.0, 844.0, 23, 4.2426406871192857, 18188.0]
[876.0, 501.0, 23, 4.2426406871192857, 20135.0]
[860.0, 922.0, 23, 4.2426406871192857, 46286.0]
[855.0, 855.0, 23, 4.2426406871192857,

[1023.0, 697.0, 25, 4.2426406871192857, 10717.0]
[946.0, 154.0, 25, 4.2426406871192857, 17985.0]
[841.0, 1018.0, 25, 4.2426406871192857, 12636.0]
[826.0, 374.0, 25, 4.2426406871192857, 41496.0]
[779.0, 294.0, 25, 4.2426406871192857, 29087.0]
[757.0, 617.0, 25, 4.2426406871192857, 24763.0]
[698.0, 487.0, 25, 4.2426406871192857, 30435.0]
[683.0, 34.0, 25, 4.2426406871192857, 13401.0]
[672.0, 514.0, 25, 4.2426406871192857, 28035.0]
[569.0, 414.0, 25, 4.2426406871192857, 30838.0]
[468.0, 533.0, 25, 4.2426406871192857, 16423.0]
[103.0, 39.0, 25, 4.2426406871192857, 16660.0]
[54.0, 0.0, 25, 8.4852813742385713, 12041.0]
[39.0, 365.0, 25, 4.2426406871192857, 26026.0]


In [36]:
"""CHECK the blobs"""

stack=3   # Change stack number to inspect across Z-stack

channel=2 # to indicate colormap for plotting
PlotCheckBlobs(OneChannel_ss, blobsAll_ss,channel, stack,)
channel=3 # to indicate colormap for plotting
PlotCheckBlobs(OneChannel_klu, blobsAll_klu, channel, stack)



In [9]:
"""Cluster the blobs across planes"""
clusters_ss=GetClustersOneChannel(blobsList_ss)
clusters_klu=GetClustersOneChannel(blobsList_klu)

Estimated number of clusters: 2008
Estimated number of clusters: 1847


In [ ]:
'''CHECK the clusters'''
# Plot the clusters in 3D
PlotCheckClusters(clusters_ss)
plt.title("channel 2",fontsize=20)

PlotCheckClusters(clusters_klu)
plt.title("Channel 3",fontsize=20) 

In [10]:
'''Find the actual position of the signals'''
signals_ss=SignalPositionsOneChannel(clusters_ss,blobs_dict_ss)
signals_klu=SignalPositionsOneChannel(clusters_klu,blobs_dict_klu)

In [37]:
'''CHECK & plot the Actual Position of the two signals'''
y_s,x_s,z_s,i_s = ToActualScale(signals_ss).T
y_k,x_k,z_k,i_k = ToActualScale(signals_klu).T
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_s, y_s, z_s,color='green')
ax.scatter(x_k, y_k, z_k,color='red')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_zlim([-1,6])
plt.title("Actual Signal Positions", fontsize=20)
plt.show()

In [12]:
'''Find the pairs between the two channels'''
# empirically, distance between 2 dots won't exceed 1.4 microns
    '''Adjust Max pair distance here'''
limit=1.4
pairs=FindPairs(signals_ss,signals_klu,limit)
print "Found {} signal pairs".format(len(pairs))

Found 1055 signal pairs


In [13]:
'''CHECK & Plot all the pairs in 3D'''
pair_coord_pic=[]# This one has pixel coord
for pair in pairs:
    pair_coord_pic.append([signals_ss[pair[0]][0:3],signals_klu[pair[1]][0:3]])

pair_coord=[]# This one has actual coord
for i in range(len(pair_coord_pic)):
     pair_coord.append(ToActualScale(pair_coord_pic[i]))
PlotPairs3D(pair_coord)
#print pair_coord_pic

In [15]:
'''calculate the distance distribution of the pairs'''
distances=[pairs[index][2] for index in range(len(pairs))]
hist, bin_edges=np.histogram(distances,bins=25)
plt.bar(bin_edges[:-1],hist,width=0.03)
plt.xlim(0, 1.4)
plt.title('Distance Distribution of the Pairs')
plt.xlabel('Unit: um')
plt.ylabel('Number of Pairs')
plt.show()

In [75]:
'''plot pairs with a threshold PROJECTED to one image with a threshold'''    
# Background Channel
channel=0
stack=14
threshold=0
# lamin channel
PlotPairsProjected(image_arrays,channel,stack,pair_coord_pic,threshold)


In [76]:
'''Plot the pairs across the planes with certain channel stack one by one'''
# Background Channel
channel=0
threshold=0.8
for stack in range(0,1):
    PlotPairsImage(image_arrays,channel, stack, pair_coord_pic, threshold)

In [78]:
'''Spread out the stacks with actual signal and a background'''

# 1. plot actual signals
# 2. plot pairs with threshold

N_rows=6
N_cols=3
resolution=20
channel=1
threshold=0
# Show the WHOLE Z-STACK

#plt.figure(figsize = (resolution*N_cols,resolution*N_rows))
plt.figure(figsize = (50,50))

gs1 = gridspec.GridSpec(N_rows, N_cols)
#gs1 = gridspec.GridSpec(2, 2)
gs1.update(wspace=0.20, hspace=0.20)

images=GetOneChannel(image_arrays,channel)

for stack in range(10):
    ax1 = plt.subplot(gs1[stack])
    plt.axis('on')
    image = images[stack]
    plt.imshow(image,cmap=colomap[channel])
    ax1.set_aspect('equal')
    
    selected=[]

    for pair in pair_coord_pic:
        #print pair
        distance=GetDistance(pair[0],pair[1])
        if distance<threshold:  # only take the pairs with distance above threshold
            continue
        if pair[0][2]>stack+0.5 or pair[0][2]<stack-0.5: # take the pairs near the stack
            continue

        selected.append(pair)

    y_s,x_s,z_s = np.array([selected[index][0] for index in range(len(selected))]).T
    y_k,x_k,z_k = np.array([selected[index][1] for index in range(len(selected))]).T

    ax1.scatter(x_s, y_s,color='green')
    ax1.scatter(x_k, y_k,color='red')
    ax1.set_xlabel('X Label')
    ax1.set_ylabel('Y Label')
    ax1.set_title("Channel {}, Stack {}".format(channel,stack),fontsize=20)

    #print log




In [79]:
#print pairs

print pair_coord_pic

print signal_ss

[[array([ 348.        ,  465.        ,    6.47599338]), array([ 347.54202202,  465.        ,    6.45797798])], [array([  75.        ,  933.        ,   21.46643531]), array([  75.53155096,  933.19375143,   21.49497333])], [array([  84.61064947,  297.38935053,   22.16124565]), array([  84.        ,  297.        ,   22.05944617])], [array([ 169.        ,  158.        ,    0.74656339]), array([ 169.        ,  157.56866328,    0.43133672])], [array([  2.60000000e+01,   1.02023621e+03,   8.54316599e-01]), array([  2.60000000e+01,   1.02000000e+03,   4.37978739e-01])], [array([ 452.        ,   78.53639939,    6.53639939]), array([ 452.        ,   79.        ,    6.13263106])], [array([ 641.        ,  422.        ,   11.98403465]), array([ 641.        ,  422.        ,   11.44773757])], [array([ 342.17691343,  548.        ,   19.6467242 ]), array([ 342.35589368,  548.        ,   19.11020329])], [array([ 688.75296546,  248.83104946,    4.62530115]), array([ 689.        ,  249.8274861 ,    4.4313

NameError: name 'signal_ss' is not defined

In [ ]:
# '''Check axes distribution of pairs on the image'''
# #print long
# y=np.array([long[index][0] for index in range(len(long))]).T
# x=np.array([long[index][1] for index in range(len(long))]).T
# z=np.array([long[index][2] for index in range(len(long))]).T
# #print y

# plt.subplot(131)
# hist, bin_edges=np.histogram(y,bins=10)
# plt.bar(bin_edges[:-1],hist,width=100)
# plt.xlim(0,1024)
# plt.title("Y Axis Position Distribution")
# plt.show()

# plt.subplot(132)
# hist, bin_edges=np.histogram(x,bins=10)
# plt.bar(bin_edges[:-1],hist,width=100)
# plt.xlim(0,1024)
# plt.title("X Axis Position Distribution")
# plt.show()

# plt.subplot(133)
# hist, bin_edges=np.histogram(z,bins=40)
# plt.bar(bin_edges[:-1],hist,width=0.05)
# plt.xlim(0,7)
# plt.title("Z Axis distance Distribution")
# plt.show()

In [12]:
'''extract & plot the pairs with distances above a threshold in 3D'''
threshold=0.8
PlotPairs3D(pair_coord,threshold)

In [16]:
'''Check pair orientation'''

y=np.array([pair_coord[index][0][0]-pair_coord[index][1][0] for index in range(len(pair_coord))]).T
x=np.array([pair_coord[index][0][1]-pair_coord[index][1][1] for index in range(len(pair_coord))]).T
z=np.array([pair_coord[index][0][2]-pair_coord[index][1][2] for index in range(len(pair_coord))]).T

plt.subplot(131)
hist, bin_edges=np.histogram(y,bins=40)
plt.bar(bin_edges[:-1],hist,width=0.05)
plt.xlim(-1.5, 1.5)
plt.axvline(0, color='k')
plt.title("Y axis distance distribution")
plt.show()

plt.subplot(132)
hist, bin_edges=np.histogram(x,bins=40)
plt.bar(bin_edges[:-1],hist,width=0.05)
plt.xlim(-1.5, 1.5)
plt.title("X axis distance distribution")
plt.axvline(0, color='k')
plt.show()

plt.subplot(133)
hist, bin_edges=np.histogram(z,bins=40)
plt.bar(bin_edges[:-1],hist,width=0.05)
plt.xlim(-1.5, 1.5)
plt.title("Z axis distance distribution")
plt.axvline(0, color='k')
plt.show()



In [17]:
'''Check where the Z plus or minus pairs are'''

z=np.array([pair_coord[index][0][2]-pair_coord[index][1][2] for index in range(len(pair_coord))]).T

selected_pairs=[]

for i in range(len(z)):
    if z[i]>0:
        selected_pairs.append(pair_coord[i])

print len(selected_pairs)
PlotPairs3D(selected_pairs)
plt.title('3D Plot of Pairs with >0 Z distance',fontsize=20)


selected_pairs=[]
for i in range(len(z)):
    if z[i]<0:
        selected_pairs.append(pair_coord[i])

print len(selected_pairs)
PlotPairs3D(selected_pairs)
plt.title('3D Plot of Pairs with <0 Z distance',fontsize=20)



873
182
